<a href="https://colab.research.google.com/github/niko-vaas/tutorial-notebooks/blob/main/FineTuningBERTUsingIMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning BERT's pretrained model

In this tutorial, we will fine-tune a BERT pretrained model (pretrained by Google) on data from [IMDB](imdb.com).

First, let's install the required packages. If you have done my prior tutorials, some of the packages are likely already installed.

In [ ]:
!pip install transformers torch datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

You may have gotten a couple of errors about conflicting or out-of-date packages. It shouldn't be a huge deal for now.

Now, let's import the important packages and sub-packages needed for fine-tuning BERT.

In [ ]:
import transformers
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader
import datasets
from datasets import load_dataset
from sklearn.model_selection import train_test_split

We'll focus on sentiment analysis, where we want to classify a piece of text as expressing positive, negative, or neutral sentiment. BERT can be fine-tuned for various tasks; this is just one example.

The `datasets` library from earlier has a variety of datasets available to train on. For now, let's load the IMDB dataset. This is what we will be using to train our custom neural net.

The dataset will automatically be split into training, unsupervised, and test sets.

In [ ]:
data = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

BERT works with numerical inputs, so we need to convert our text data into numbers. This process is called **tokenization**. If you've tried my previous BERT tutorial before, you may remember tokenization.

We'll use a tokenizer specifically designed for BERT.

Because a validation set is not already generated by `datasets.load_dataset()`, we will use the `unsupervised` dataset.

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

def preprocess_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

train_data = data["train"].map(preprocess_function, batched=True)
val_data = data["unsupervised"].map(preprocess_function, batched=True)
test_data = data["test"].map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

This process may take a while: IMDB has a large dataset, so tokenization will also take a while.

Since this model is pre-trained, we don't need to retrain every single layer. Let's freeze all the layers except for the last few, since we will be training the final few layers.

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=3) # 3 labels (positive, negative, neutral)

for param in model.bert.parameters():
  param.requires_grad = False  # Freeze pre-trained weights

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now, we train.

Regardless of what software you use, training will take a long, long time.

In [ ]:
print(data['train'][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)  # Optimizer
loss_fn = torch.nn.CrossEntropyLoss()  # Loss function for classification
num_epochs = 3  # Number of training epochs
batch_size = 16  # Batch size

def train_model(model, optimizer, loss_fn, data, num_epochs):
  """
  Trains the sentiment analysis model.

  Args:
      model: The BERT model to be trained.
      optimizer: The optimizer used for updating model weights.
      loss_fn: The loss function used for calculating training loss.
      data: The dataloader containing training data.
      num_epochs: The number of training epochs.
  """
  # for epoch in range(num_epochs):
  #   running_loss = 0.0
  for batch in data['train']:
      optimizer.zero_grad()
      # Access data within the batch (assuming keys "text" and "label")
      text = batch["text"]
      print(text)
      labels = batch["label"]
      outputs = model.forward(input_ids=text, labels=labels)  # Pass data by argument names
      loss = loss_fn(outputs.logits, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()

  # model.train()
  # for epoch in range(num_epochs):
  #   for batch in data:
  #     optimizer.zero_grad()  # Clear gradients before each batch
  #     outputs = model(**batch)
  #     loss = loss_fn(outputs.logits, batch["label"])
  #     loss.backward()
  #     optimizer.step()
      # You can add code to print training progress here (e.g., epoch and loss)

def evaluate_model(model, loss_fn, val_dataloader):

  """
  Evaluates the sentiment analysis model on the validation set.

  Args:
      model: The trained model.
      loss_fn: The loss function used for calculating validation loss.
      val_dataloader: The dataloader containing validation data.

  Returns:
      The average validation loss.
  """
  model.eval()
  total_loss = 0
  with torch.no_grad():
    for batch in val_dataloader:
      outputs = model(**batch)
      loss = loss_fn(outputs.logits, batch["label"])
      total_loss += loss.item()
  avg_loss = total_loss / len(val_dataloader)
  print(f"Validation Loss: {avg_loss:.4f}")
  return avg_loss

# Example usage
train_model(model, optimizer, loss_fn, data, num_epochs=3)
validation_loss = evaluate_model(model, loss_fn, val_dataloader)

UnboundLocalError: local variable 'text' referenced before assignment